# SQLAlchemy Core: Establishing Connectivity

In [1]:
from sqlalchemy import create_engine


engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)
# 'sqlite' -- what kind of DB we communicate.
# 'pysqlite' -- what kind of DBAPI we use. If omitted, SQLAlchemy will use a default DBAPI.
# '/:memory:' -- how we locate the database.

# SQLAlchemy Core: Working with Transactions and the DBAPI

In [2]:
from sqlalchemy import text


with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2024-10-02 12:31:31,725 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-02 12:31:31,727 INFO sqlalchemy.engine.Engine select 'hello world'
2024-10-02 12:31:31,729 INFO sqlalchemy.engine.Engine [generated in 0.00245s] ()
[('hello world',)]
2024-10-02 12:31:31,730 INFO sqlalchemy.engine.Engine ROLLBACK


In [3]:
# 'commit as you go'
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

2024-10-02 12:33:41,182 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-02 12:33:41,183 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2024-10-02 12:33:41,185 INFO sqlalchemy.engine.Engine [generated in 0.00248s] ()
2024-10-02 12:33:41,186 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-10-02 12:33:41,187 INFO sqlalchemy.engine.Engine [generated in 0.00063s] [(1, 1), (2, 4)]
2024-10-02 12:33:41,189 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
# 'begin once'
# this block do NOT require .commit() in the end, because 
# it will ROLLBACK if an exception was raised
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2024-10-02 12:38:31,869 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-02 12:38:31,870 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-10-02 12:38:31,871 INFO sqlalchemy.engine.Engine [cached since 290.7s ago] [(6, 8), (9, 10)]
2024-10-02 12:38:31,872 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
# 'begin once'
# with ERROR
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 45, "ye": 32}, {"x": 9, "y": 10, "z": 11}],
    )

2024-10-02 12:40:16,050 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-02 12:40:16,051 INFO sqlalchemy.engine.Engine ROLLBACK


StatementError: (sqlalchemy.exc.InvalidRequestError) A value is required for bind parameter 'y'
[SQL: INSERT INTO some_table (x, y) VALUES (?, ?)]
[parameters: [{'x': 45, 'ye': 32}, {'x': 9, 'y': 10, 'z': 11}]]
(Background on this error at: https://sqlalche.me/e/20/cd3x)

### Tuple Assignment


In [8]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for x, y in result:
        print(f"x: {x}  y: {y}")

2024-10-02 12:50:03,515 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-02 12:50:03,516 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2024-10-02 12:50:03,517 INFO sqlalchemy.engine.Engine [cached since 342.7s ago] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
x: 45  y: 32
x: 9  y: 10
2024-10-02 12:50:03,519 INFO sqlalchemy.engine.Engine ROLLBACK


### Integer Index

In [9]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row[0]}  y: {row[1]}")

2024-10-02 12:50:36,352 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-02 12:50:36,353 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2024-10-02 12:50:36,354 INFO sqlalchemy.engine.Engine [cached since 375.6s ago] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
x: 45  y: 32
x: 9  y: 10
2024-10-02 12:50:36,355 INFO sqlalchemy.engine.Engine ROLLBACK


### Attribute Name

In [10]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2024-10-02 12:51:04,054 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-02 12:51:04,055 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2024-10-02 12:51:04,055 INFO sqlalchemy.engine.Engine [cached since 403.3s ago] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
x: 45  y: 32
x: 9  y: 10
2024-10-02 12:51:04,058 INFO sqlalchemy.engine.Engine ROLLBACK


### Mapping Access

In [11]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for dict_row in result.mappings():
        print(f"x: {dict_row['x']}  y: {dict_row['y']}")

2024-10-02 12:52:02,839 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-02 12:52:02,839 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2024-10-02 12:52:02,840 INFO sqlalchemy.engine.Engine [cached since 462.1s ago] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
x: 45  y: 32
x: 9  y: 10
2024-10-02 12:52:02,841 INFO sqlalchemy.engine.Engine ROLLBACK


## Sending Parameters

In [12]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2024-10-02 12:53:50,613 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-02 12:53:50,614 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2024-10-02 12:53:50,615 INFO sqlalchemy.engine.Engine [generated in 0.00186s] (2,)
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
x: 45  y: 32
x: 9  y: 10
2024-10-02 12:53:50,616 INFO sqlalchemy.engine.Engine ROLLBACK


In [16]:
# NEVER DO IT IN SUCH a WAY!!!!
# it prevents SQL injection attacks

y_lim = 4

with engine.connect() as conn:
    result = conn.execute(text(f"SELECT x, y FROM some_table WHERE y > {y_lim}"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2024-10-02 12:57:35,115 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-02 12:57:35,116 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > 4
2024-10-02 12:57:35,116 INFO sqlalchemy.engine.Engine [cached since 41.14s ago] ()
x: 6  y: 8
x: 9  y: 10
x: 45  y: 32
x: 9  y: 10
2024-10-02 12:57:35,118 INFO sqlalchemy.engine.Engine ROLLBACK


## Sending Multiple Parameters

In [17]:
# ""executemany"" -- the name of this style of execution 
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}],
    )
    conn.commit()

2024-10-02 13:00:21,011 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-02 13:00:21,012 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-10-02 13:00:21,013 INFO sqlalchemy.engine.Engine [cached since 1600s ago] [(11, 12), (13, 14)]
2024-10-02 13:00:21,014 INFO sqlalchemy.engine.Engine COMMIT


# Execution with an ORM Session

In [18]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2024-10-02 13:17:00,395 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-02 13:17:00,396 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2024-10-02 13:17:00,396 INFO sqlalchemy.engine.Engine [generated in 0.00076s] (6,)
x: 6  y: 8
x: 9  y: 10
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
x: 45  y: 32
2024-10-02 13:17:00,398 INFO sqlalchemy.engine.Engine ROLLBACK


In [19]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}],
    )
    session.commit()

2024-10-02 13:32:06,096 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-02 13:32:06,097 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2024-10-02 13:32:06,098 INFO sqlalchemy.engine.Engine [generated in 0.00090s] [(11, 9), (15, 13)]
2024-10-02 13:32:06,099 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2024-10-02 13:33:01,827 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-02 13:33:01,828 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2024-10-02 13:33:01,828 INFO sqlalchemy.engine.Engine [cached since 961.4s ago] (6,)
x: 6  y: 8
x: 9  y: 11
x: 9  y: 11
x: 11  y: 12
x: 13  y: 15
x: 45  y: 32
2024-10-02 13:33:01,830 INFO sqlalchemy.engine.Engine ROLLBACK
